Apache Spark is used in this notebook to improve the speed of data processing, 

In [3]:
%pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Test Spark installation
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.sql("select 'spark' as hello")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:
%conda install -c cyclus java-jdk

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Read all_data.csv
spark=SparkSession.builder.master('local')\
                  .appName("SparkTimeSeries")\
                  .getOrCreate()

df=spark.read.option("header",True)\
                .csv("all_data.csv")

df.printSchema()

root
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- symbol: string (nullable = true)



In [8]:
df.show(n=5)

+------------------+------------------+------------------+------------------+------------------+----------+------+
|              Open|              High|               Low|             Close|         Adj Close|    Volume|symbol|
+------------------+------------------+------------------+------------------+------------------+----------+------+
|142.99000549316406| 146.4499969482422| 142.1199951171875|145.49000549316406|145.28819274902344|71185600.0|  AAPL|
| 144.0800018310547| 148.9499969482422|            143.25|148.47000122070312|148.26405334472656|78140700.0|  AAPL|
|149.77999877929688|150.86000061035156| 148.1999969482422| 150.1699981689453|149.96170043945312|76259900.0|  AAPL|
|150.74000549316406|151.57000732421875| 146.6999969482422|147.07000732421875|146.86599731445312|81420900.0|  AAPL|
| 147.9199981689453|151.22999572753906|146.91000366210938|             151.0|150.79054260253906|82982400.0|  AAPL|
+------------------+------------------+------------------+------------------+---

In [24]:
############################################################ Data cleaning ##########################################
from pyspark.sql.functions import desc
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import countDistinct

# Verify # of ticker symbols
Num_symbols=df.select(countDistinct("symbol"))
Num_symbols.show()

# Count no. of Null in each col
Count_null=df.select([count(when(isnan(c)| col(c).isNull()| (df[c] == 0),c)).alias(c) for c in df.columns])
Count_null.show()


+----------------------+
|count(DISTINCT symbol)|
+----------------------+
|                   100|
+----------------------+

+----+----+---+-----+---------+------+------+
|Open|High|Low|Close|Adj Close|Volume|symbol|
+----+----+---+-----+---------+------+------+
|   0|   0|  0|    0|        0|     0|     0|
+----+----+---+-----+---------+------+------+

